In [ ]:
#Importation Dataset CIFAR-10
import tensorflow as tf

(x_train_t, y_train_t), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()


In [ ]:
#Librairie
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout


In [ ]:
#Split le data set en train et en validation (pour les CNN)

N_train_t = np.shape(x_train_t)[0]
N_train = round(0.8*N_train_t) # 80% pour le train et 20% pour la validation
x_train,y_train = x_train_t[0:N_train],y_train_t[0:N_train]
x_val,y_val = x_train_t[N_train:],y_train_t[N_train:]

#Normalisation des images

x_train, x_val, x_test = x_train / 255.0, x_val / 255.0, x_test / 255.0

plt.figure
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.imshow(x_train[i],cmap='gray')
    plt.axis('off')
plt.show()

In [ ]:
#Algo de ML : Decision Tree 

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

x_train_flat = x_train_t.reshape((x_train_t.shape[0], -1))
x_test_flat = x_test.reshape((x_test.shape[0], -1))

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_flat)
x_test_scaled = scaler.transform(x_test_flat)

arbre = DecisionTreeClassifier(max_depth=10, random_state=42)
arbre.fit(x_train_scaled, y_train_t.ravel())

param_grid = {
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 10, 20],
    'criterion': ['gini', 'entropy']
}

# GridSearch avec validation croisée 3-fold
grid_search = GridSearchCV(arbre, param_grid, cv=3)
grid_search.fit(x_train_scaled, y_train_t.ravel())

y_pred = arbre.predict(x_test_scaled)
acc = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='macro')  # Calcul du rappel
print(f"Précision de l'arbre de décision sur CIFAR-10 : {acc:.2f}")
print(f"Rappel de l'arbre de décision sur CIFAR-10 : {recall:.2f}")

In [ ]:
#Foret Aleatoire 
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Entraînement du modèle
rf.fit(x_train_scaled, y_train_t.ravel())

# Prédiction et évaluation
y_pred = rf.predict(x_test_scaled)
acc = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='macro')  # Calcul du rappel
print(f"Précision de la forêt aléatoire sur CIFAR-10 : {acc:.2f}")
print(f"Rappel de la forêt aléatoire sur CIFAR-10 : {recall:.2f}")

In [ ]:
#3 CNN aux architectures différentes

# Redimensionner les images (28x28 -> 28x28x1) pour la convolution
x_train = x_train.reshape(-1, 32, 32, 3)
x_val = x_val.reshape(-1, 32, 32, 3)
x_test = x_test.reshape(-1, 32, 32, 3)
# Encodage One-Hot des labels
y_train_c = to_categorical(y_train, 10)
y_val_c = to_categorical(y_val, 10)
y_test_c = to_categorical(y_test, 10)

In [ ]:
#CNN classique (comme dans le TD)
model = Sequential([
Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)), # Couche Conv
MaxPooling2D(2,2), # Max Pooling
Conv2D(64, (3,3), activation='relu'), # Deuxième Couche Conv
MaxPooling2D(2,2),
Flatten(), # Aplatit les données
Dense(128, activation='relu'), # Couche Fully Connected
Dense(10, activation='softmax') # Couche de sortie (10 classes)
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
#CNN un peu plus complexe (une couche de convolution en + et ajout de dropout)

model_2 = Sequential([
Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)), 
MaxPooling2D(2,2), 
Dropout(0.25),
Conv2D(64, (3,3), activation='relu'), 
MaxPooling2D(2,2),
Dropout(0.25),
Conv2D(128, (3,3), activation='relu'),
Flatten(),
Dense(256, activation='relu'), 
Dense(10, activation='softmax') 
])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_2.summary()

In [ ]:
#Modèle basé sur le CDNN https://arxiv.org/pdf/1202.2745v1 


model_3 = Sequential([
    Conv2D(300, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),  
    Conv2D(300, (2, 2), activation='relu'),  
    MaxPooling2D((2, 2)), 
    Conv2D(300, (2, 2), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(300, (2, 2), activation='relu'), 
    MaxPooling2D((2, 2)),
    Flatten(), 
    Dense(300, activation='relu'),  
    Dense(100, activation='relu'),
    Dense(10, activation='softmax')  
])


model_3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_3.summary()

In [ ]:
# Entraînement du 1er model
history_1 = model.fit(
    x_train, y_train_c,
    validation_data=(x_val, y_val_c),
    epochs=10,
    batch_size=64,
    callbacks=[early_stopping]
)


In [ ]:
# Entraînement model_2
history_intermediate = model_2.fit(
    x_train, y_train_c,
    validation_data=(x_val, y_val_c),
    epochs=10,
    batch_size=64,
    callbacks=[early_stopping]
)

In [ ]:

# Entraînement du CDNN
history_deep = model_3.fit(
    x_train, y_train_c,
    validation_data=(x_val, y_val_c),
    epochs=10,
    batch_size=64,
    callbacks=[early_stopping]
)

In [ ]:
'''plt.plot(history_1.history['loss'], label='Train Loss CNN Simple')
plt.plot(history_1.history['val_loss'], label='Validation Loss CNN Simple')
plt.plot(history_1.history['accuracy'], label='Train Accuracy CNN Simple')
plt.plot(history_1.history['val_accuracy'], label='Validation Accuracy CNN Simple')
plt.plot(history_intermediate.history['loss'], label='Train Loss CNN Inter')
plt.plot(history_intermediate.history['val_loss'], label='Validation Loss CNN Inter')
plt.plot(history_intermediate.history['accuracy'], label='Train Accuracy CNN Inter')
plt.plot(history_intermediate.history['val_accuracy'], label='Validation Accuracy CNN Inter')'''
plt.plot(history_deep.history['loss'], label='Train Loss CDNN')
plt.plot(history_deep.history['val_loss'], label='Validation Loss CDNN')
plt.plot(history_deep.history['accuracy'], label='Train Accuracy CDNN')
plt.plot(history_deep.history['val_accuracy'], label='Validation Accuracy CDNN')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()